In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

%matplotlib widget

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [8]:
df = pd.read_csv("../archive/song_data.csv")
df.drop_duplicates(subset='song_name', keep='first', inplace=True)
df = df.drop('song_name', axis=1)
df = df.drop('loudness', axis=1)

In [9]:
X, y = df.drop(['song_popularity'],axis=1), df['song_popularity']
X

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,262333,0.005520,0.496,0.682,0.000029,8,0.0589,1,0.0294,167.060,4,0.4740
1,216933,0.010300,0.542,0.853,0.000000,3,0.1080,0,0.0498,105.256,4,0.3700
2,231733,0.008170,0.737,0.463,0.447000,0,0.2550,1,0.0792,123.881,4,0.3240
3,216933,0.026400,0.451,0.970,0.003550,0,0.1020,1,0.1070,122.444,4,0.1980
4,223826,0.000954,0.447,0.766,0.000000,10,0.1130,1,0.0313,172.011,4,0.5740
...,...,...,...,...,...,...,...,...,...,...,...,...
18827,114582,0.898000,0.370,0.136,0.000263,7,0.0999,1,0.0433,146.081,4,0.0592
18830,159645,0.893000,0.500,0.151,0.000065,11,0.1110,1,0.0348,113.969,4,0.3000
18832,182211,0.847000,0.719,0.325,0.000000,0,0.1250,1,0.0355,130.534,4,0.2860
18833,352280,0.945000,0.488,0.326,0.015700,3,0.1190,1,0.0328,106.063,4,0.3230


In [10]:
X, y = df.drop('song_name',axis=1).to_numpy(), df['song_popularity'].to_numpy()

In [11]:
X

array([[7.30000e+01, 2.62333e+05, 5.52000e-03, ..., 1.67060e+02,
        4.00000e+00, 4.74000e-01],
       [6.60000e+01, 2.16933e+05, 1.03000e-02, ..., 1.05256e+02,
        4.00000e+00, 3.70000e-01],
       [7.60000e+01, 2.31733e+05, 8.17000e-03, ..., 1.23881e+02,
        4.00000e+00, 3.24000e-01],
       ...,
       [2.30000e+01, 1.82211e+05, 8.47000e-01, ..., 1.30534e+02,
        4.00000e+00, 2.86000e-01],
       [5.50000e+01, 3.52280e+05, 9.45000e-01, ..., 1.06063e+02,
        4.00000e+00, 3.23000e-01],
       [6.00000e+01, 1.93533e+05, 9.11000e-01, ..., 9.14900e+01,
        4.00000e+00, 5.81000e-01]])

In [6]:
my_regression_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', MLPRegressor(max_iter=500, random_state=0)) # Hidden layers are by default relu, output layer activation function is 'identity' -> linear
])

param_grid = {
    'model__hidden_layer_sizes': [(100,50,20), (50, 10, 5, 2), (200, 100, 50, 20), (400,200,100,50)],
    'model__alpha': [0.01, 0.03, 0.1, 0.3, 1.0],
    'model__learning_rate_init': [0.0001, 0.0003, 0.001, 0.003, 0.01],
}

grid_search = GridSearchCV(my_regression_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X, y)

print("Best Hyperparameters:", grid_search.best_params_)

c:\Users\Emile Peeters\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Emile Peeters\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Emile Peeters\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Emile Peeters\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Emile Peeters\anaconda3\Lib\sit

Best Hyperparameters: {'model__alpha': 0.01, 'model__hidden_layer_sizes': (12, 8, 4), 'model__learning_rate_init': 0.001}


In [ ]:
results = grid_search.pivot(index='param_alpha', columns='param_learning_rate_init', values='mean_test_score')
sns.heatmap(results, annot=True, fmt=".3f", cmap="viridis")
plt.show()

In [7]:
print("Mean Test Scores:")
print(grid_search.cv_results_['mean_test_score'])

best_index = grid_search.best_index_
print(f"\nBest Hyperparameters Index: {best_index}")

print(f"\nAverage Performance of Best Hyperparameters: {grid_search.cv_results_['mean_test_score'][best_index]}")

Mean Test Scores:
[ -474.72847263  -449.51045212  -333.21603511  -408.92334481
  -408.74037342 -2805.00908219 -2308.57411347 -1327.93130929
  -408.72851462  -409.28575271 -2776.98057589 -2777.49357265
 -1722.79401305  -574.11375972  -409.23270699 -2722.31138159
 -2721.69952241 -1389.62341844  -416.39033517  -408.70941919
  -472.69760537  -449.47563933  -333.18816302  -794.98705574
  -486.84932879 -2805.00908219 -1153.99393667  -114.45155788
  -317.12512304  -409.28582725 -2781.8945221  -2783.16503046
 -2783.69339234 -1475.63898668  -409.23384023 -2722.31138159
 -2721.69952241 -1389.62329449  -416.39025595  -408.70941093
  -472.69882155  -449.47697496  -333.21123105  -778.51948647
  -486.81488433 -2805.00908219 -1092.84352962  -463.29611886
  -278.63808279  -409.28585772 -2781.89448257 -2783.16507252
 -2783.69996382 -1874.50292654  -409.23402582 -2722.31138159
 -2721.69952241 -1389.62329452  -416.38366045  -408.7087516
  -472.70178131  -449.47829943  -333.19726172  -735.86233417
  -510.

In [11]:
best_model = grid_search.best_estimator_.named_steps['model']
print(f"Number of inputs:  {best_model.n_features_in_}")
print(f"Number of outputs: {best_model.n_outputs_}")
print(f"Number of layers:  {best_model.n_layers_}")
print(f"Layer sizes: {[l.shape for l in best_model.coefs_]}")

Number of inputs:  14
Number of outputs: 1
Number of layers:  5
Layer sizes: [(14, 12), (12, 8), (8, 4), (4, 1)]
